# LAB 7: Transfer Learning - Ejemplo

**Ingeniería Electrónica**

**Inteligencia Artificial**

**15/06/2022**

**Transferir el aprendizaje** permite tomar un pequeño conjunto de datos y producir un modelo preciso. Este método utiliza grandes redes que fueron entrenadas durante mucho tiempo en grandes conjuntos de datos, transfiriendo ese conocimiento a nuestra propia red. Para esto utilizaremos la API de [Weights & Biases](https://wandb.ai/site), por lo que primero es necesario registrarse en W&B y crear un proyecto con el nombre "transfer_learn".

Reutilizaremos una red para detección de alimentos (_groceries_), usando el script `groceries.py` que desarga el dataset de [Freiburg](https://github.com/PhilJd/freiburg_groceries_dataset)

In [ ]:
#!pip install wandb -qqq
import wandb

In [ ]:
# Iniciar sesión en su cuenta W&B
wandb.login()

In [ ]:
import os
os.environ["WANDB_ENTITY"] = "mlclass"

In [ ]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet import ResNet50, decode_predictions, preprocess_input
from keras.utils import np_utils
import groceries
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = groceries.load_data()

In [ ]:
# echar un vistazo a los tipos de imágenes que estamos tratando
plt.imshow(x_train[50].astype(int))

In [ ]:
# Imprime las clases objetivo
class_names

In [ ]:
# comprobar qué tan balanceada es la distribución de clases
plt.hist(y_train_raw)

In [ ]:
# Codificar las salidas
y_train = keras.utils.np_utils.to_categorical(y_train_raw)
y_test = keras.utils.np_utils.to_categorical(y_test_raw)

In [ ]:
# Cargar ResNet50 entrenada en imagenet
resnet_model = ResNet50(weights="imagenet")

In [ ]:
resnet_model.summary()

In [ ]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('IMAGEN.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = resnet_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
# Deberíamos preprocesar las imágenes de la misma manera que se preprocesaron las imágenes resnet
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [ ]:
# Crear un nuevo modelo que sea ResNet50 menos la última capa
last_layer = resnet_model.get_layer("avg_pool")

resnet_layers = keras.Model(inputs=resnet_model.inputs, outputs=last_layer.output)
resnet_layers.summary()

In [ ]:
# Podemos unir directamente los modelos

new_model=Sequential()
new_model.add(resnet_layers)
new_model.add(Dense(25, activation="sigmoid"))

new_model.layers[0].trainable=False

new_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

new_model.summary()

In [ ]:
# Podemos permitir que algunas de las capas de resnet cambien mientras entrenamos.

new_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in new_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in new_model.layers[0].layers[-11:]:
    layer.trainable = True
    
new_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#wandb.init(project="transfer_learn")
#wandb.init(project="transfer_learn", settings=wandb.Settings(start_method="thread"))
wandb.init(project="transfer_learn", entity="USUARIO")
new_model.fit(x_train_preprocessed, y_train, epochs=50, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])